In [ ]:
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

# Build a graph

In [ ]:
tf.reset_default_graph()

arrays = [tf.constant(np.arange(50*i, 50*(i + 1))) for i in range(100)]
batch_size = 8
dataset_size = 100*50


dataset = tf.data.Dataset.from_tensor_slices(arrays)
num_shards = len(arrays)


# read the files in parallel
dataset = dataset.shuffle(buffer_size=num_shards)
dataset = dataset.apply(tf.contrib.data.parallel_interleave(
    lambda x: tf.data.Dataset.from_tensor_slices(x), cycle_length=10
))
dataset = dataset.prefetch(buffer_size=batch_size)


dataset = dataset.shuffle(buffer_size=125)
dataset = dataset.repeat(3)


dataset = dataset.apply(tf.contrib.data.map_and_batch(
    lambda x: x, batch_size=batch_size,
    num_parallel_batches=1, drop_remainder=False
))
dataset = dataset.prefetch(buffer_size=1)


iterator = tf.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)
init = iterator.make_initializer(dataset)
numbers = iterator.get_next()

In [ ]:
num_epochs = 1
result = []
with tf.Session() as sess:
    for i in range(num_epochs):
        sess.run(init)
        try:
            while True:
                output = sess.run(numbers)
                result.extend(list(output))
        except tf.errors.OutOfRangeError:
            pass

In [ ]:
len(result)

In [ ]:
_, counts = np.unique(np.array(result), return_counts=True)

In [ ]:
plt.hist(result, bins=100);

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(result, 'r.');